#### This notebook performs regression using a Convolutional Neural Net on molecular descriptors.
#### There are 1119 molecular descriptors for each molecule. A dummy zero descriptor is added to make the total number of descriptors 1120.
#### Then the descriptors are reshaped into 35 by 32 (35*32)  grayscale images.
#### Using Leave one out cross-validation data generated by create_dataset.ipynb, models are trained on different folds of training data to regress on pIC50 values.
#### The trained models are saved in models_info folder.
#### Various parameters like R2, tropsha's criteria etc are also evaluated for each model and saved in models_info/logs/logs.txt
#### Please note that the above mentioned folders are empty. They will be populated once this notebook is run.
#### Best models based on R2 test are saved in best_models folder. (.json files are model architecture and .h5 files re model weights) Performance parameters for best chosen models are available in best_models/performance_parameters_of_best_models/performance_parameters.xlsx. 
#### They were evaluated through XternalValidationPlus (https://sites.google.com/site/dtclabxvplus/) by passing the test set predictions of each model. These predictions are available in models_info/data_to_evaluate_xternal_validation_parameters

In [ ]:
import pandas as pd
import numpy as np
import sys
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D,UpSampling2D, Dropout
from keras.callbacks import EarlyStopping,ModelCheckpoint
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import math
import pickle
import logging

In [ ]:
# Evaluate tropsha's criteria
def tropsha(predicted_holdout_values,holdout_labels):
    
    
    # Find attributes of test set
    reg = LinearRegression().fit(np.reshape(predicted_holdout_values,(-1,1)),holdout_labels)
    

    r2 = reg.score(np.reshape(predicted_holdout_values,(-1,1)),holdout_labels)
    
    reg = LinearRegression(fit_intercept = False).fit(np.reshape(predicted_holdout_values,(-1,1)),holdout_labels)

    r02 = reg.score(np.reshape(predicted_holdout_values,(-1,1)),holdout_labels)
    
    k = reg.coef_

    r0_dash2 = reg.score(np.reshape(predicted_holdout_values,(-1,1)),np.reshape(np.array(holdout_labels),(-1,1)))

    k_dash = reg.coef_
    
    print("R2: (>0.6): "+str(r2))
    print("R02: "+str(r02))
    print("R0_dash2: "+str(r0_dash2))
    print("R02-R0_dash2: (<0.3): "+str(np.abs(r02 - r0_dash2)))
    print("k: (0.85,1.15): "+str(k))
    print("(R2-R02)/R2: (<0.1)"+str((r2-r02)/r2))
    print("k_dash: (0.85,1.15): "+str(k_dash))
    print("(R2-R0_dash2)/R2 (<0.1): "+str((r2-r0_dash2)/r2))
    if (r2 > 0.6 and np.abs(r02 - r0_dash2)<0.3 and k>=0.85 and k<=1.15 and (r2-r02)/r2<0.1 and k_dash>=0.85 and k_dash<=1.15
        and (r2-r0_dash2)/r2<0.1):
        return True
    else:
        return False



In [ ]:
# Read test set
csv_df_test = pd.read_csv('data/test_compounds.csv')
csv_df_test

In [ ]:
# output folder for saving models and weights
out_folder_name = "models_info/"

# Log location for logging model performance
logging.basicConfig(filename=out_folder_name+"logs/log.txt", level=logging.INFO)

# No of models due to data folds generated by Leave-one-out cross validation. In this notebook there are 69 datapoints 
# in the training set. Hence n_models = 69
n_models = 94-len(csv_df_test)
# Image dimensions for molecular descriptors reshaping
img_height = 35
img_width = 32

# Load scaler used to reshape training data. This will be used to inverse transform data to get actual pIC50 values. 
# Note that the data used to train CNN is already min-max scaled between 0 and 1.
scaler = pickle.load(open('scaler_data/scaler.dat','rb'))
test_data = csv_df_test.loc[:,'nAcid':'values']

# min max inverse transform for test data
test_data_inverse_transform_actual = scaler.inverse_transform(test_data)
test_values_inverse_transform_actual = test_data_inverse_transform_actual[:,len(test_data_inverse_transform_actual[0])-1]
test_data = test_data.loc[:,'nAcid':'Zagreb']
test_labels = np.reshape(np.array(csv_df_test['values']),-1)
num_of_test_examples = len(test_labels)

# Add a dummy 0 to increase descriptor columns to 1120 so that descriptors can be reshaped into 35 by 32 grayscale images.
test_data['zero'] = 0
test_imgs = np.reshape(np.array(test_data),(num_of_test_examples,img_height,img_width,1))

r2_training_list = []
r2_validation_list = []
dict_of_metrics = {}

# looping and creating models
for i in range(0,n_models):
    # Read training data
    csv_df_training = pd.read_csv('data/training_set_'+str(i)+'.csv')
    # Read leave one out data
    csv_df_loo = pd.read_csv('data/loo_set__'+str(i)+'.csv')

    training_data = csv_df_training.loc[:,'nAcid':'values']
    loo_data = csv_df_loo.loc[:,'nAcid':'values']
    num_of_training_examples = len(training_data)
    num_of_loo_examples = len(loo_data)
    
    # Minmax scaler inverse transform for  train/loo set
    training_data_inverse_transform_actual = scaler.inverse_transform(training_data)
    loo_data_inverse_transform_actual = scaler.inverse_transform(loo_data)
    
    training_values_inverse_transform_actual = training_data_inverse_transform_actual[:,len(training_data_inverse_transform_actual[0])-1]
    loo_values_inverse_transform_actual = loo_data_inverse_transform_actual[:,len(loo_data_inverse_transform_actual[0])-1]
    ######
    training_labels = np.reshape(np.array(training_data['values']),-1)
    loo_labels = loo_data['values']
    
    training_data = training_data.loc[:,'nAcid':'Zagreb']
    loo_data = loo_data.loc[:,'nAcid':'Zagreb']
    
    # Add a dummy 0 to increase descriptor columns to 1120 so that descriptors can be reshaped into 35 by 32 grayscale images.
    training_data['zero'] = 0
    loo_data['zero'] = 0
    
    training_imgs = np.reshape(np.array(training_data),(num_of_training_examples,img_height,img_width,1))
    loo_imgs = np.reshape(np.array(loo_data),(num_of_loo_examples,img_height,img_width,1))
    
    # CNN
    #create model
    model = Sequential()
    #add model layers
    model.add(Conv2D(4, kernel_size=5, activation='relu', input_shape=(img_height,img_width,1),padding='valid'))
    model.add(Dropout(0.02))
    model.add(Conv2D(6, kernel_size=5, activation='relu',padding='valid'))
    model.add(Dropout(0.04))
    model.add(Conv2D(8, kernel_size=5, activation='relu',padding='valid'))
    model.add(Dropout(0.06))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    
    # Check point for saving best model
    mc = ModelCheckpoint(out_folder_name+"/models/model_"+str(i)+".h5", monitor='val_loss', mode='min', verbose=0, save_best_only=True,
                         save_weights_only=True)
      
    
    print("Model-"+str(i))
    logging.info("Model-"+str(i))
    
    # train the model
    history = model.fit(training_imgs, training_labels,
                        validation_data=(test_imgs,test_labels), 
                        epochs=2000,batch_size=25,verbose = 0,callbacks=[mc])
    training_loss_list = history.history['loss']
    validation_loss_list = history.history['val_loss']
    
     # serialize model to JSON
    model_json = model.to_json()
    with open(out_folder_name+"/models/model_"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    
    # Load  model
    json_file = open(out_folder_name+'/models/model_'+str(i)+'.json')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(out_folder_name+'/models/model'+str(i)+'.h5')
    
    # Predict training set
    predicted_train_values = loaded_model.predict(training_imgs)
    # min max scaler inverse transform for predicted train values
    training_data_predicted = training_data.loc[:,'nAcid':'Zagreb']
    training_data_predicted['values'] = predicted_train_values
    training_data_inverse_transform_predicted = scaler.inverse_transform(training_data_predicted)
    training_value_inverse_transform_predicted = training_data_inverse_transform_predicted[:,len(training_data_inverse_transform_predicted[0])-1]
    
    # Predict loo set
    predicted_loo_values = loaded_model.predict(loo_imgs)
    # min max scaler inverse transform for predicted loo values
    loo_data_predicted = loo_data.loc[:,'nAcid':'Zagreb']
    loo_data_predicted['values'] = predicted_loo_values
    loo_data_inverse_transform_predicted = scaler.inverse_transform(loo_data_predicted)
    loo_value_inverse_transform_predicted = loo_data_inverse_transform_predicted[:,len(loo_data_inverse_transform_predicted[0])-1]
    
    # Predict test set
    predicted_test_values = loaded_model.predict(test_imgs)
    test_data_predicted = test_data.loc[:,'nAcid':'Zagreb']
    test_data_predicted['values'] = predicted_test_values
    # min max scaler inverse transform for predicted test values
    test_data_inverse_transform_predicted = scaler.inverse_transform(test_data_predicted)
    test_value_inverse_transform_predicted = test_data_inverse_transform_predicted[:,len(test_data_inverse_transform_predicted[0])-1]
    
    predicted_test_values = np.reshape(predicted_test_values,-1)
    
    # Calculate model r2
    r2_training = r2_score(training_values_inverse_transform_actual,np.reshape(training_value_inverse_transform_predicted,-1))
    r2_test = r2_score(test_values_inverse_transform_actual,test_value_inverse_transform_predicted)
    
    # Calculate LOO-Q2
    avg_predicted_train_values = np.mean(training_value_inverse_transform_predicted)
    avg_train_values = np.mean(np.array(training_values_inverse_transform_actual))
    TSS = np.mean(np.square(np.reshape(training_values_inverse_transform_actual,-1)-avg_train_values))
    PRESS = np.square(np.reshape(loo_value_inverse_transform_predicted,-1)-np.reshape(loo_values_inverse_transform_actual,-1))
    LOO_Q2 = 1-(PRESS/TSS)
    
    # Calculate Q2-ext-F1
    q2_ext_numerator = np.sum(np.square(test_value_inverse_transform_predicted - np.reshape(np.array(test_values_inverse_transform_actual),-1)))
    q2_ext_denominator = np.sum(np.square(np.reshape(np.array(test_values_inverse_transform_actual),-1) - avg_train_values))
    q2_ext = 1-(q2_ext_numerator/q2_ext_denominator)
     
    mae_test = mean_absolute_error(test_values_inverse_transform_actual, test_value_inverse_transform_predicted)
    se_training = math.sqrt(mean_squared_error(training_values_inverse_transform_actual,training_value_inverse_transform_predicted))
    mae_training = mean_absolute_error(training_values_inverse_transform_actual, training_value_inverse_transform_predicted)
    max_training_response_value = np.max(training_values_inverse_transform_actual)
    min_training_response_value = np.min(training_values_inverse_transform_actual)
    
    print("Training response avg: "+str(avg_train_values))
    logging.info("Training response avg: "+str(avg_train_values))
    print("Training response range: "+str(max_training_response_value-min_training_response_value))
    logging.info("Training response range: "+str(max_training_response_value-min_training_response_value))
    print("R2 Training: "+str(r2_training))
    logging.info("R2 Training: "+str(r2_training))
    print("R2 Test (>0.7): "+str(r2_test))
    logging.info("R2 Test (>0.7): "+str(r2_test))
    print("LOO-Q2 (>0.7) : "+str(float(LOO_Q2)))
    logging.info("LOO-Q2 (>0.7) : "+str(float(LOO_Q2)))
    print("Q2-ext-F1 (>0.7): "+str(q2_ext))
    logging.info("Q2-ext-F1 (>0.7): "+str(q2_ext))
    print("SE Training: "+str(se_training))
    logging.info("SE Training: "+str(se_training))
    print("MAE Training: "+str(mae_training))
    logging.info("MAE Training: "+str(mae_training))
    print("MAE Test: "+str(mae_test))
    logging.info("MAE Test: "+str(mae_test))
    tropsha_pass = tropsha(test_value_inverse_transform_predicted,test_values_inverse_transform_actual)
    if tropsha_pass:
        print("Tropsha's Criteria: "+"Pass")
        logging.info("Tropsha's Criteria: "+"Pass")
    else:
        print("Tropsha's Criteria: "+"Fail")
        logging.info("Tropsha's Criteria: "+"Fail")
    
    # Save test outputs to evaluate MAE criteria. XternalValidationPlus can be used to evaluate MAE criteria.(https://sites.google.com/site/dtclabxvplus/)
    data = [test_values_inverse_transform_actual]
    data.append(test_value_inverse_transform_predicted)
    data = np.transpose(data)
    data_df = pd.DataFrame(data)
    # For each fold of data Yobs and Ypred for test compounds is written to the following csv file. These files can be used to provide input to 
    # XternalValidationPlus (https://sites.google.com/site/dtclabxvplus/) to evaluate external validation parameters.
    data_df.to_csv(str(out_folder_name)+"/data_to_evaluate_xternal_validation_parameters/data_df"+str(i)+".csv",header=['Yobs','Ypred'],index=False)

    if (tropsha_pass and r2_test>0.7 and float(LOO_Q2)>0.7 and q2_ext>0.7):
        
        
        print("Net: "+"Pass")
    else:
        print("Net: "+"Fail")
        print("########")
        logging.info("########")